# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [26]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [27]:
# Load csv data into dataframe
data="../output_data/cities_data.csv"
data_df=pd.read_csv(data)
data_df

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
0,Bowen,-20.02,148.23,73.40,73,40,5.82,AU,1601028206
1,Burē,10.70,37.07,70.83,69,47,3.38,ET,1601028206
2,Ürümqi,43.80,87.60,60.80,40,17,13.42,CN,1601028207
3,Terrasini,38.15,13.08,88.00,48,29,23.04,IT,1601028207
4,Clyde River,70.47,-68.59,28.53,96,100,7.36,CA,1601028207
...,...,...,...,...,...,...,...,...,...
664,Grandview,38.89,-94.53,63.00,77,1,8.05,US,1601028393
665,Monte Alegre,-2.01,-54.07,75.16,84,17,7.70,BR,1601028393
666,Cleveland,41.50,-81.70,62.60,72,1,6.93,US,1601028143
667,Port Macquarie,-31.43,152.92,71.01,33,0,1.01,AU,1601028394


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [28]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [29]:
# Create a poverty Heatmap layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(data_df[["Lat","Lng"]], weights=data_df["Humidity"], 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [30]:
# Narrow down the cities to fit weather conditions
# Drop cities with a maximum temp above 100 F and less than 70 F
ideal_df=data_df.drop(data_df[data_df['Max_Temp'] > 100].index , inplace=False)
ideal_df=ideal_df.drop(ideal_df[ideal_df['Max_Temp'] < 70].index , inplace=False)
# Drop cities with wind speed greater than 20 mph
ideal_df=ideal_df.drop(ideal_df[ideal_df['Wind_Speed']>20].index, inplace=False)
# Drop citis with cloudiness greater than 70
ideal_df=ideal_df.drop(ideal_df[ideal_df['Cloudiness']>70].index, inplace=False)
ideal_df

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
0,Bowen,-20.02,148.23,73.40,73,40,5.82,AU,1601028206
1,Burē,10.70,37.07,70.83,69,47,3.38,ET,1601028206
8,Thoen,17.61,99.22,83.53,86,68,3.27,TH,1601028208
9,Natal,-5.79,-35.21,73.40,94,40,4.70,BR,1601027932
10,Khani,41.96,42.96,89.60,38,10,12.75,GE,1601028209
...,...,...,...,...,...,...,...,...,...
662,Mount Isa,-20.73,139.50,78.80,14,0,6.93,AU,1601028392
663,Abéché,13.83,20.83,87.80,26,20,9.53,TD,1601028392
665,Monte Alegre,-2.01,-54.07,75.16,84,17,7.70,BR,1601028393
667,Port Macquarie,-31.43,152.92,71.01,33,0,1.01,AU,1601028394


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [32]:
#hotel_df["Hotel Name"]=pd.DataFrame
#city_country=ideal_df["City"],["Country"]
#city_country

(0               Bowen
 1                Burē
 8               Thoen
 9               Natal
 10              Khani
             ...      
 662         Mount Isa
 663            Abéché
 665      Monte Alegre
 667    Port Macquarie
 668              Fare
 Name: City, Length: 207, dtype: object,
 ['Country'])

In [33]:
# Set api url
url="https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

# Define parameters

location=[]
locations=[]
names=[]
city=[]
country=[]
address=[]


key = g_key
radius = "5000"
type = "lodging"
keyword = "hotel"

for l,m in zip(ideal_df["Lat"], ideal_df["Lng"]):
   location.append(f"{l},{m}")

for loc in location[0:5]:
    
    query_url = url + "location="+ loc + "&radius=" + radius + "&type=" + "lodging" + "&keyword=" + keyword + "&key="+key
    try:
        print("Retreiving data")
    
        response=requests.get(query_url)
        response_json=response.json()
        results=response_json['results']
        names.append(results[0]['name'])
        locations.append(results[0]['geometry']['location'])
        #address.append((results[0]['plus_code']['compound_code']).split(","))
        #city.append(ideal_df["Ctiy"]address[2])
        #country.append(address[3])
     
    except:
        print("Data not found")
    
    #params = {
#    "location": ideal_df[["Lat","Lng"]], 
#    "rankby": "distance",
#    "type": "lodging",
#    "key": gkey,

# Define query_url
#
pprint(response_json)

Retreiving data
Retreiving data
Retreiving data
Retreiving data
Retreiving data
{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 42, 'lng': 42.9999999},
                           'viewport': {'northeast': {'lat': 42.00134982989273,
                                                      'lng': 43.00134972989272},
                                        'southwest': {'lat': 41.99865017010729,
                                                      'lng': 42.99865007010727}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png',
              'name': 'Nino village',
              'place_id': 'ChIJrU0qIkTPXEARj7-qSWaFMnY',
              'plus_code': {'compound_code': '2222+22 Sakraula, Georgia',
                            'global_code': '8HJ52222+22'},
              'rating': 0,
              'reference': 'ChIJrU0qIkTPXEARj7-qSWaFMnY',
              'scope': 'GOOGLE',
              '

In [34]:
locations

[{'lat': -19.9792147, 'lng': 148.2597912},
 {'lat': 10.7102662, 'lng': 37.0818826},
 {'lat': 17.6311173, 'lng': 99.23927739999999},
 {'lat': -5.7803515, 'lng': -35.1935076},
 {'lat': 42, 'lng': 42.9999999}]

In [35]:
names


['Whitsunday Sands Resort',
 'Meseret Hotel',
 'Nakornthoen Hotel',
 'Porto Suites Natal Hotel',
 'Nino village']

In [24]:
address_list[1]


IndexError: list index out of range

In [ ]:
locations_df=pd.DataFrame(locations)
hotel_data={"Hotel Name":names,"Address":address}
data_df=pd.DataFrame(hotel_data, columns=["Hotel Name","Address"])
Locations_df=locations_df.rename(columns={"lat":"Lat","lng":"Lng"})
hotel_df=data_df.join(Locations_df)
hotel_info=hotel_data
#locations_df
hotel_df


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
